In [4]:
import pandas as pd

df = pd.read_csv("/Users/giovanni-lunetta/stat_3255/stat_3255_final/data/test.csv")

In [5]:
import pandas as pd
import numpy as np
df.drop(['Unique Key', 'Location', 'Agency Name'], axis=1, inplace=True)
df.drop(['Bridge Highway Segment', 'Road Ramp', 'Bridge Highway Direction', 'Bridge Highway Name', 'Taxi Pick Up Location', 'Taxi Company Borough', 'Vehicle Type', 'Due Date', 'Facility Type'], axis=1, inplace=True)
df.drop(['Zip Codes', 'Community Districts', 'Borough Boundaries', 'City Council Districts'], axis=1, inplace=True)
df.drop(['Street Name', 'BBL', 'X Coordinate (State Plane)', 'Y Coordinate (State Plane)', 'Status', 'Community Board', 'Cross Street 1', 'Cross Street 2', 'Intersection Street 1', 'Intersection Street 2', 'Landmark', 'Park Borough'], axis=1, inplace=True)
df.dtypes
df.describe()
count_low = df[df['Incident Zip'] <= 10000.000000]['Incident Zip'].count()
count_high = df[df['Incident Zip'] >= 11697]['Incident Zip'].count()
print(count_low)
print(count_high)
df.drop(df[df['Incident Zip'] <= 10000.000000].index, inplace=True)
df.drop(df[df['Incident Zip'] >= 11697].index, inplace=True)
# fill null values with 'Other'
df['Location Type'] = df['Location Type'].fillna('Other')

# change all 'Other (Explain Below)' to 'Other' because they are the same thing
df['Location Type'] = df['Location Type'].replace('Other (Explain Below)', 'Other')

# fill null values with 'Other'
df['Descriptor'] = df['Descriptor'].fillna('Other')

# change all versions of other to uniform 'Other'
df['Descriptor'] = df['Descriptor'].replace({'Other (Explain Below)': 'Other',
                                             'Other (complaint details)': 'Other',
                                             'Other/Unknown': 'Other'})

df["Address Type"].value_counts(dropna=False)

# fill null values with 'UNRECOGNIZED'
df['Address Type'] = df['Address Type'].fillna('UNRECOGNIZED')

df['Park Facility Name'] = df['Park Facility Name'].fillna('Unspecified')

df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
df['Resolution Action Updated Date'] = pd.to_datetime(df['Resolution Action Updated Date'])
df['Created Date'] = pd.to_datetime(df['Created Date'])
df['Closed Date'] = pd.to_datetime(df['Closed Date'], errors='coerce')
# Check if there are any instances where the "Closed Date" is earlier than the "Created Date"
closed_earlier = df[df['Closed Date'] < df['Created Date']]

if closed_earlier.empty:
    print("No instances where 'Closed Date' is earlier than 'Created Date'")
else:
    print("There are instances where 'Closed Date' is earlier than 'Created Date'")
# creates an index containing all of examples where closed date is before created data and drops those columns
index_to_drop = df.index[df['Closed Date'] < df['Created Date']].tolist()
df.drop(index_to_drop, inplace=True)
print((df.isnull().sum() * 100)/ len(df))
df.dropna(subset=['Latitude', 'Longitude'], inplace=True)
df['duration'] = df['Closed Date'] - df['Created Date']

# convert the resulting timedelta object to a numerical value in seconds
df['duration_seconds'] = df['duration'].dt.total_seconds()

df['duration_hours'] = df['duration'].dt.total_seconds() / 3600

df['duration_days'] = df['duration'].dt.days

# Create two new columns - one for the day of the week and one for the type of day (weekday/weekend):
df['day_of_week'] = df['Created Date'].dt.dayofweek
df['day_type'] = np.where(df['day_of_week'] < 5, 'weekday', 'weekend')
df['duration_hours'] = df['duration_hours'].fillna(168)
# from uszipcode import SearchEngine
# import pandas as pd

# sr = SearchEngine()

# borough_zipcodes = {
#     'Bronx': [str(zipcode) for zipcode in range(10451, 10476)],
#     'Brooklyn': [str(zipcode) for zipcode in range(11201, 11257)],
#     'Manhattan': [str(zipcode) for zipcode in range(10001, 10283)],
#     'Queens': [str(zipcode) for zipcode in range(11004, 11110)] + [str(zipcode) for zipcode in range(11351, 11698)],
#     'Staten Island': [str(zipcode) for zipcode in range(10301, 10315)]
# }

# zip_info_list = []

# for borough, zipcodes in borough_zipcodes.items():
#     for zipcode in zipcodes:
#         zip_info = sr.by_zipcode(zipcode)
#         if zip_info is not None:
#             zip_info_dict = zip_info.__dict__
#             zip_info_dict['borough'] = borough
#             zip_info_list.append(zip_info_dict)

# df1 = pd.DataFrame(zip_info_list)

# df1.to_csv('zip_codes.csv')
df_zip = pd.read_csv("/Users/giovanni-lunetta/stat_3255/stat_3255_final/data/zip_codes.csv")
df_zip.head()
df_zip.drop(['Unnamed: 0', '_sa_instance_state', 'lat', 'water_area_in_sqmi', 'bounds_east', 'zipcode_type', 'lng', 'bounds_north', 'common_city_list', 'county', 'bounds_west', 'state', 'land_area_in_sqmi', 'borough', 'timezone', 'bounds_south', 'major_city', 'radius_in_miles', 'post_office_city', 'area_code_list'], axis=1, inplace=True)
df_zip.head()
# merge df and df_zip on 'Incident Zip' in df and 'zipcode' in df_zip
df = pd.merge(df, df_zip, left_on='Incident Zip', right_on='zipcode', how='left')

# drop the 'zipcode' column from the merged dataframe
df = df.drop('zipcode', axis=1)
df = df[df['Borough'] != 'unspecified']
print((df.isnull().sum() * 100)/ len(df))
# Replace all strings containing 'noise' with 'Noise'
df['Complaint Type'] = df['Complaint Type'].str.replace(r'.*noise.*', 'noise', regex=True)
# descriptor_counts = df['Complaint Type'].value_counts()
# num_single_counts = (descriptor_counts == 1).sum()
# print(f'There are {num_single_counts} values in the Descriptor column with a value count of 1')
# descriptor_value_counts = df['Descriptor'].value_counts()

# descriptor_stats = descriptor_value_counts.describe()

# print(descriptor_stats)
counts = df['Descriptor'].value_counts()
to_remove = counts[counts < 10].index
df = df[~df['Descriptor'].isin(to_remove)]

counts2 = df['Complaint Type'].value_counts()
to_remove2 = counts2[counts2 < 10].index
df = df[~df['Complaint Type'].isin(to_remove2)]
df['Borough'].value_counts()

32
9
There are instances where 'Closed Date' is earlier than 'Created Date'
Created Date                      0.000000
Closed Date                       5.468265
Agency                            0.000000
Complaint Type                    0.000000
Descriptor                        0.000000
Location Type                     0.000000
Incident Zip                      0.953703
Incident Address                  4.224979
Address Type                      0.000000
City                              3.910732
Resolution Description            0.860525
Resolution Action Updated Date    0.747250
Borough                           0.000000
Open Data Channel Type            0.000000
Park Facility Name                0.000000
Latitude                          1.624219
Longitude                         1.624219
Police Precincts                  1.652538
dtype: float64
Created Date                      0.000000
Closed Date                       5.461553
Agency                            0.000000
Compla

brooklyn         32965
bronx            23952
queens           23772
manhattan        22077
staten island     4030
Name: Borough, dtype: int64

In [ ]:
df['Location Type'].value_counts()

counts2 = df['Location Type'].value_counts()
to_remove2 = counts2[counts2 < 10].index
df = df[~df['Location Type'].isin(to_remove2)]

df = df.dropna()

df.drop(['duration_days', 'duration_seconds', 'duration', 'Longitude', 'Latitude', 'Park Facility Name', 'Open Data Channel Type', 'Agency', 'Closed Date'], axis=1, inplace=True)

df['Incident Zip'] = df['Incident Zip'].astype(str)
df['day_of_week'] = df['day_of_week'].astype(str)
df['day_type'] = df['day_type'].astype(str)

# df_cleaned.dtypes

df['Created Date'] = pd.to_datetime(df['Created Date'])
df['Resolution Action Updated Date'] = pd.to_datetime(df['Resolution Action Updated Date'])

# Extract hour, day, and month from 'Created Date' column
df['Created Hour'] = df['Created Date'].dt.hour
df['Created Day'] = df['Created Date'].dt.day
df['Created Month'] = df['Created Date'].dt.month

# Extract hour, day, and month from 'Resolution Action Updated Date' column
df['Resolution Updated Hour'] = df['Resolution Action Updated Date'].dt.hour
df['Resolution Updated Day'] = df['Resolution Action Updated Date'].dt.day
df['Resolution Updated Month'] = df['Resolution Action Updated Date'].dt.month

df.drop(['Created Date', 'Resolution Action Updated Date'], axis=1, inplace=True)

from sklearn.preprocessing import StandardScaler

# Create a StandardScaler object
scaler = StandardScaler()

# select the variables to scale
variables_to_scale = ['housing_units', 'occupied_housing_units', 'median_home_value', 'population',
                      'median_household_income', 'population_density', 'Police Precincts', 'duration_hours', 'Created Day', 'Created Hour', 'Resolution Updated Hour', 'Resolution Updated Day']

# apply the scaler to the selected variables
df.loc[:, variables_to_scale] = scaler.fit_transform(df.loc[:, variables_to_scale])

df.drop(['Incident Zip', 'Incident Address'], axis=1, inplace=True)

df.drop(['Police Precincts', 'City'], axis=1, inplace=True)

# create a list of column names with object type
columns = ['Complaint Type', 'Descriptor', 'Location Type',
           'Address Type', 'Resolution Description', 'day_of_week', 'day_type']

# get dummies with one column removed
df = pd.get_dummies(df, columns=columns, drop_first=True)

# Define a dictionary to map borough names to numerical values
borough_dict = {'brooklyn': 0, 'bronx': 1, 'queens': 2, 'manhattan': 3, 'staten island': 4}

# Replace the string values with numerical values
df['Borough'] = df['Borough'].replace(borough_dict)

In [ ]:
df.to_csv('/Users/giovanni-lunetta/stat_3255/stat_3255_final/data/cleaned_test.csv', index=False)